In [8]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np 
import tflearn
import tensorflow as tf
import random
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [9]:
with open('prompts.json') as file:
    data = json.load(file)

In [ ]:
with open('data.pickle', 'rb') as file:
    words, labels, training, output = pickle.load(file)

In [10]:
stemmer = LancasterStemmer()

words = []
labels = []
docs_x = []
docs_y = []

for prompt in data['intents']:
    for pattern in prompt['patterns']:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        docs_x.append(tokens)
        docs_y.append(prompt['tag'])

        if prompt['tag'] not in labels:
            labels.append(prompt['tag'])

words = [stemmer.stem(w.lower()) for w in words if w != '?']
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = list(np.zeros(len(labels)))

for x, doc in enumerate(docs_x):
    bag = []

    stemmed = [stemmer.stem(w) for w in doc]

    for w in words:
        if w in stemmed:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)    
    
with open('data.pickle', 'wb') as file:
    pickle.dump((words, labels, training, output), file)

training =  np.array(training)
output = np.array(output)

In [11]:
training = np.array(training)
output = np.array(output)

In [12]:
len(output[0])

6

In [14]:
# Run this cell to create a new model
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')


Training Step: 3999  | total loss: 0.06795 | time: 0.025s
| Adam | epoch: 1000 | loss: 0.06795 - acc: 0.9960 -- iter: 24/26
Training Step: 4000  | total loss: 0.06131 | time: 0.032s
| Adam | epoch: 1000 | loss: 0.06131 - acc: 0.9964 -- iter: 26/26
--
INFO:tensorflow:C:\Users\owner\Documents\DSJC\nlp-project\scripted_bot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
# Run this cell to load a previously created model
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.load('model.tflearn')


In [15]:
def bag_of_words(s, words, stemmer):
    bag = [0 for i in range(len(words))]
    bag = list(np.zeros(len(words)))
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i]=1
    return np.array(bag)

In [16]:
def clear(): 
    from os import system, name
    # for windows 
    if name == 'nt': 
        _ = system('cls') 
    # for mac and linux(here, os.name is 'posix') 
    else: 
        _ = system('clear') 

In [17]:
def chat():

    clear()

    print('Welcome to chat')

    stemmer = LancasterStemmer()

    while True:
        inp = input("You: ")
        if inp.lower() in ['quit', 'exit', 'stop']:
            break

        results = model.predict([bag_of_words(inp, words, stemmer)])
        
        result_index = np.argmax(results)
        
        tag = labels[result_index]
        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tg['responses']
                
        print('Bot: ', random.choice(responses))

In [18]:
chat()

Welcome to chat


You:  hello there


Bot:  Hi there, how can I help?


You:  when are you open?


Bot:  We are open 7am-4pm Monday-Friday!


You:  what do you sell?


Bot:  We sell chocolate chip cookies for $2!


You:  that's all I needed goodbye


Bot:  Talk to you later


You:  quit
